In [179]:
import pandas as pd
import numpy as np
import math
from scipy import signal
import matplotlib.pyplot as plt

In [210]:
path_hilbert = 'C:/EEG_Embedded_Systems/DatosProcesados_V2/'
path_hilbert_rojo = path_hilbert + 'RawData_Outliers/7Hz'
path_hilbert_azul = path_hilbert + 'RawData_Outliers/9Hz'
path_hilbert_verde = path_hilbert + 'RawData_Outliers/11Hz'
path_hilbert_morado = path_hilbert + 'RawData_Outliers/13Hz'
path_hilbert_baseline = path_hilbert + 'RawData_Outliers/Baseline' 

In [211]:
num_total_rojo = 748
num_total_azul = 754
num_total_verde = 756
num_total_morado = 759
num_total_baseline = 759

In [212]:
colores = [num_total_rojo, num_total_azul, num_total_verde, num_total_morado, num_total_baseline]
color = ['rojo','azul','verde', 'morado', 'baseline']
paths = [path_hilbert_rojo, path_hilbert_azul, path_hilbert_verde, path_hilbert_morado, path_hilbert_baseline]
electrodos = ['EEG.O1', 'EEG.O2']

In [213]:
def leerEtiquetas(color):
  if color == 'azul':
    etiqueta = 2
  elif color == 'rojo':
    etiqueta = 1
  elif color == 'verde':
    etiqueta = 3
  elif color == 'morado':
    etiqueta = 4
  elif color == 'baseline':
    etiqueta = 5
  return etiqueta

In [214]:
def leerArchivo(path, num_archivo):
    datos = pd.read_csv(path + '/muestra' + num_archivo + '.csv')
    return datos

In [215]:
def valorAbsolutoMedio(path, num_archivo, num_total_archivos, columna):
    lista = []
    datos = leerArchivo(path, num_archivo)
    return abs((datos[columna])).to_frame().mean()[columna]
    

In [216]:
def valorAbsolutoPromedioWeightsI(path, num_archivo, num_total_archivos, columna):
  lista = []
  datos = leerArchivo(path, num_archivo)
  for fila in range(len(datos[columna])):
    fila += 1
    if fila <= 0.75*len(datos[columna]) and fila >= 0.25*len(datos[columna]):
      w = 1
    else:
      w = 0.5
    lista.append(abs(datos.iloc[fila-1][columna])*w)
  return np.array(lista).mean()

In [217]:
def valorAbsolutoPromedioWeightsII(path, num_archivo, num_total_archivos, columna):
  lista = []
  datos = leerArchivo(path, num_archivo)
  for fila in range(len(datos[columna])):
    fila += 1
    if fila <= 0.75*len(datos[columna]) and fila >= 0.25*len(datos[columna]):
      w = 1
    elif fila < 0.25*len(datos[columna]) :
      w = 4*fila/len(datos[columna])
    else:
      w = 4*(len(datos[columna])-fila)/len(datos[columna])
    lista.append(abs(datos.iloc[fila-1][columna])*w)
  return np.array(lista).mean()

In [218]:
def detectorRegistro(path, num_archivo, num_total_archivos, columna):
  datos = leerArchivo(path, num_archivo)
  return math.exp(np.log(abs(datos[columna]).to_frame()).mean()[columna])

In [219]:
def ValorAbsolutoMediana(path, num_archivo, num_total_archivos, columna):
  datos = leerArchivo(path, num_archivo)
  return abs((datos[columna])).to_frame().median()[columna]
  

In [220]:
def varianza(path, num_archivo, num_total_archivos, columna):
  datos = leerArchivo(path, num_archivo)
  return abs((datos[columna])).var(ddof=0)

In [221]:
def diferencia_absoluta(v):
  sum_diff = 0
  for i,x in enumerate(v):
    if i+1 < len(v):
      sum_diff += abs(v[i+1] - x)
      r = sum_diff/len(v)
  return r
def valorMediaDiferenciaAbsoluta(path, num_archivo, num_total_archivos, columna):
  datos = leerArchivo(path, num_archivo)
  return diferencia_absoluta(datos[columna].to_list())

In [222]:
def zeroCrossing(path, num_archivo, num_total_archivos, columna):
  datos = leerArchivo(path, num_archivo)[columna].to_list()
  zerocrossing = 0
  for i in range(1, len(datos)):
    if ((datos[i-1]) > 0 and datos[i] < 0):
        zerocrossing += 1
    if ((datos[i-1]) < 0 and datos[i] > 0):
        zerocrossing += 1
  return zerocrossing

In [223]:
def frecuenciaMedia(path, num_archivo, num_total_archivos, columna):
  datos = leerArchivo(path, num_archivo)[columna].to_list()
  frecuencia, spectrum = signal.periodogram(datos, 128)
  frecuenciaMedia = ((frecuencia * spectrum).sum())/(spectrum.sum())
  return frecuenciaMedia

In [224]:
def frecuenciaPico(path, num_archivo, num_total_archivos, columna):
  datos = leerArchivo(path, num_archivo)[columna].to_list()
  f, sp = signal.periodogram(datos, 128)
  frecuencia = f[30:60]
  spectrum = sp[30:60]
  frecuencia_pico_indice = np.argmax(spectrum)
  return frecuencia[frecuencia_pico_indice]

In [225]:
def VarianzaFrecuenciaCentral(path, num_archivo, num_total_archivos, columna):
    datos = leerArchivo(path, num_archivo)[columna].to_list()
    f, sp = signal.periodogram(datos, 128)
    frecuencia_promedio = np.mean(f)
    return np.sum(sp * pow(f- frecuencia_promedio,2))/frecuencia_promedio

In [226]:
def PSDmax(path, num_archivo, num_total_archivos, columna):
  datos = leerArchivo(path, num_archivo)
  datos = datos[columna].to_list()
  f, sp = signal.periodogram(datos, 128)
  frecuencia = f[30:60]
  spectrum = sp[30:60]
  psd_max = np.max(spectrum)
  return psd_max

In [227]:
electrodo = electrodos[0]
muestra = []
df1 = pd.DataFrame()
for i in range(12):
  lista = []
  if i==0:
    for i in range(len(colores)):
      print(colores[i])
      for num_archivo in range(colores[i]+1):
        muestra.append(num_archivo)
        lista.append(valorAbsolutoMedio(paths[i], str(num_archivo), colores[i], electrodo))
    df1['valorAbsMedio'] = lista
    lista = []
  elif i==1:
    for i in range(len(colores)):
      for num_archivo in range(colores[i]+1):
        lista.append(valorAbsolutoPromedioWeightsI(paths[i], str(num_archivo), colores[i], electrodo))
    df1['valorAbsPromWI'] = lista
    lista = []
  elif i==2:
    for i in range(len(colores)):
      for num_archivo in range(colores[i]+1):
        lista.append(valorAbsolutoPromedioWeightsII(paths[i], str(num_archivo), colores[i], electrodo))
    df1['valorAbsPromWII'] = lista
    lista = []
  elif i==3:
    for i in range(len(colores)):
      for num_archivo in range(colores[i]+1):
        lista.append(detectorRegistro(paths[i], str(num_archivo), colores[i], electrodo))
    df1['LogDet'] = lista
    lista = []
  elif i==4:
    for i in range(len(colores)):
      for num_archivo in range(colores[i]+1):
        lista.append(ValorAbsolutoMediana(paths[i], str(num_archivo), colores[i], electrodo))
    df1['valorAbsMediana'] = lista
    lista = []
  elif i==5:
    for i in range(len(colores)):
      for num_archivo in range(colores[i]+1):
        lista.append(varianza(paths[i], str(num_archivo), colores[i], electrodo))
    df1['Varianza'] = lista
    lista = []
  elif i==6:
    for i in range(len(colores)):
      for num_archivo in range(colores[i]+1):
        lista.append(valorMediaDiferenciaAbsoluta(paths[i], str(num_archivo), colores[i], electrodo))
    df1['valorMediaDiferenciaAbs'] = lista
    lista = []
  elif i==7:
    for i in range(len(colores)):
      for num_archivo in range(colores[i]+1):
        lista.append(zeroCrossing(paths[i], str(num_archivo), colores[i], electrodo))
    df1['zeroCrosing'] = lista
    lista = []
  elif i==8:
    for i in range(len(colores)):
      for num_archivo in range(colores[i]+1):
        lista.append(frecuenciaMedia(paths[i], str(num_archivo), colores[i], electrodo))
    df1['frecMedia'] = lista
    lista = []
  elif i==9:
    for i in range(len(colores)):
      for num_archivo in range(colores[i]+1):
        lista.append(frecuenciaPico(paths[i], str(num_archivo), colores[i], electrodo))
    df1['Frecuencia_en_PSDMAX'] = lista
    lista = []
  elif i==10:
    for i in range(len(colores)):
      for num_archivo in range(colores[i]+1):
        lista.append(VarianzaFrecuenciaCentral(paths[i], str(num_archivo), colores[i], electrodo))
    df1['VarianzaFrecCentral'] = lista
    lista = []
  elif i==11:
    for i in range(len(colores)):
      for num_archivo in range(colores[i]+1):
        lista.append(PSDmax(paths[i], str(num_archivo), colores[i], electrodo))
    df1['PSD_max'] = lista
    lista = []


748
754
756
759
759


In [228]:
df1

,valorAbsMedio,valorAbsPromWI,valorAbsPromWII,LogDet,valorAbsMediana,Varianza,valorMediaDiferenciaAbs,zeroCrosing,frecMedia,Frecuencia_en_PSDMAX,VarianzaFrecCentral,PSD_max
0,4188.344319,3141.106091,3143.860811,4188.312863,4191.538574,263.003064,4.907199,0,4.110897,10.832962,24188.660444,8.691949
1,4160.571140,3117.119715,3118.722716,4160.556085,4159.487305,125.300125,5.124539,0,8.796557,16.105960,8754.741189,12.768647
2,4165.060963,3122.079979,3124.524351,4165.046551,4166.794922,120.029040,5.020855,0,8.545238,13.968820,8604.462317,9.049548
3,4167.631013,3125.788802,3126.146217,4167.616336,4167.820313,122.336343,10.596902,0,25.076336,11.766816,2695.099529,12.115849
4,4161.946576,3120.916378,3120.923287,4161.930249,4163.077148,135.850221,9.297143,0,18.309750,15.360000,6125.559513,5.251169
...,...,...,...,...,...,...,...,...,...,...,...,...
3776,4154.579238,3118.003925,3115.557700,4154.572461,4153.974121,56.307669,2.859525,0,5.374231,10.022371,4954.615523,1.941834
3777,4165.831746,3122.231869,3124.601907,4165.827770,4167.820313,33.107793,2.451898,0,7.027914,10.262806,2693.225769,1.473546
3778,4168.033107,3123.843807,3126.229870,4168.030201,4168.589844,24.219221,2.382774,0,9.410979,9.122494,1713.190552,2.837485
3779,4165.732984,3124.475396,3124.403545,4165.717913,4166.154053,125.553248,10.885044,0,25.533001,14.791111,2963.298884,19.618981


In [229]:
  #DEFINIR LOS INTERVALOS DE FRECUENCIA
  interv1 = [0,6.5]
  interv2 = [interv1[1], interv1[1]+6.5]
  interv3 = [interv2[1], interv2[1]+6.5]
  interv4 = [19.5, 26]
  interv5 = [interv4[1], interv4[1]+6.5]
  interv6 = [interv5[1], interv5[1]+6.5]
  interv7 = [interv6[1], interv6[1]+6.5]
  interv8 = [interv7[1], interv7[1]+6.5]
  interv9 = [interv8[1], interv8[1]+6.5]
  interv10 = [interv9[1], interv9[1]+6.5]
  
  primer_intervalo = []
  segundo_intervalo = []
  tercer_intervalo = []
  cuarto_intervalo = []
  quinto_intervalo = []
  sexto_intervalo = []
  septimo_intervalo = []
  octavo_intervalo = []
  noveno_intervalo = []
  decimo_intervalo = []

In [230]:
def HistogramaAmplitud(path, num_archivo, columna):
  interv1_lista = []
  interv2_lista = []
  interv3_lista = []
  interv4_lista = []
  interv5_lista = []
  interv6_lista = []
  interv7_lista = []
  interv8_lista = []
  interv9_lista = []
  interv10_lista = []
  datos = leerArchivo(path, num_archivo)[columna]
  frecuencia, spectrum = signal.periodogram(datos, 128)
  datos = frecuencia
  spectrum = spectrum.tolist()
  total = sum(spectrum)
  for dato in datos:
    if interv1[0] <= dato < interv1[1]:
      interv1_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv2[0] <= dato < interv2[1]:
      interv2_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv3[0] <= dato < interv3[1]:
      interv3_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv4[0] <= dato < interv4[1]:
      interv4_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv5[0] <= dato < interv5[1]:
      interv5_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv6[0] <= dato < interv6[1]:
      interv6_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv7[0] <= dato < interv7[1]:
      interv7_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv8[0] <= dato < interv8[1]:
      interv8_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv9[0] <= dato < interv9[1]:
      interv9_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv10[0] <= dato < interv10[1]:
      interv10_lista.append(spectrum[int(np.where(datos == dato)[0])])
  primer_intervalo.append(sum(interv1_lista)/total)
  segundo_intervalo.append(sum(interv2_lista)/total)
  tercer_intervalo.append(sum(interv3_lista)/total)
  cuarto_intervalo.append(sum(interv4_lista)/total)
  quinto_intervalo.append(sum(interv5_lista)/total)
  sexto_intervalo.append(sum(interv6_lista)/total)
  septimo_intervalo.append(sum(interv7_lista)/total)
  octavo_intervalo.append(sum(interv8_lista)/total)
  noveno_intervalo.append(sum(interv9_lista)/total)
  decimo_intervalo.append(sum(interv10_lista)/total)

In [231]:
for i in range(len(colores)):
  for num_archivo in range(colores[i]+1):
    HistogramaAmplitud(paths[i], str(num_archivo), electrodo)

In [232]:
df1['FHist1'] = primer_intervalo
df1['FHist2'] = segundo_intervalo
df1['FHist3'] = tercer_intervalo
df1['FHist4'] = cuarto_intervalo
df1['FHist5'] = quinto_intervalo
df1['FHist6'] = sexto_intervalo
df1['FHist7'] = septimo_intervalo
df1['FHist8'] = octavo_intervalo
df1['FHist9'] = noveno_intervalo
df1['FHist10'] = decimo_intervalo

In [233]:
etiquetas = []
for i in range(len(colores)):
  for num_archivo in range(colores[i]+1):
    etiquetas.append(leerEtiquetas(color[i]))

In [234]:
df1['Etiqueta'] = etiquetas
df1['Muestra'] = muestra

In [235]:
df1

,valorAbsMedio,valorAbsPromWI,valorAbsPromWII,LogDet,valorAbsMediana,Varianza,valorMediaDiferenciaAbs,zeroCrosing,frecMedia,Frecuencia_en_PSDMAX,...,FHist3,FHist4,FHist5,FHist6,FHist7,FHist8,FHist9,FHist10,Etiqueta,Muestra
0,4188.344319,3141.106091,3143.860811,4188.312863,4191.538574,263.003064,4.907199,0,4.110897,10.832962,...,0.053020,0.029086,0.013134,0.016074,0.004940,0.000622,0.000199,0.000233,1,0
1,4160.571140,3117.119715,3118.722716,4160.556085,4159.487305,125.300125,5.124539,0,8.796557,16.105960,...,0.199357,0.039454,0.030395,0.028012,0.011137,0.000551,0.000257,0.000037,1,1
2,4165.060963,3122.079979,3124.524351,4165.046551,4166.794922,120.029040,5.020855,0,8.545238,13.968820,...,0.171154,0.033623,0.032266,0.039550,0.013038,0.000600,0.000251,0.000114,1,2
3,4167.631013,3125.788802,3126.146217,4167.616336,4167.820313,122.336343,10.596902,0,25.076336,11.766816,...,0.076596,0.182252,0.179425,0.260489,0.072303,0.007158,0.001904,0.000501,1,3
4,4161.946576,3120.916378,3120.923287,4161.930249,4163.077148,135.850221,9.297143,0,18.309750,15.360000,...,0.074356,0.154374,0.129906,0.166674,0.057366,0.005268,0.001543,0.000701,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3776,4154.579238,3118.003925,3115.557700,4154.572461,4153.974121,56.307669,2.859525,0,5.374231,10.022371,...,0.037347,0.047350,0.028244,0.030348,0.013062,0.000962,0.000353,0.000193,5,755
3777,4165.831746,3122.231869,3124.601907,4165.827770,4167.820313,33.107793,2.451898,0,7.027914,10.262806,...,0.056380,0.038795,0.045317,0.051246,0.008163,0.001762,0.000483,0.000147,5,756
3778,4168.033107,3123.843807,3126.229870,4168.030201,4168.589844,24.219221,2.382774,0,9.410979,9.122494,...,0.071389,0.080169,0.055828,0.040231,0.023089,0.003106,0.001376,0.000692,5,757
3779,4165.732984,3124.475396,3124.403545,4165.717913,4166.154053,125.553248,10.885044,0,25.533001,14.791111,...,0.090239,0.107977,0.218857,0.253044,0.117418,0.006533,0.002050,0.000668,5,758


In [236]:
df1.to_csv("C:/EEG_Embedded_Systems/New_Features/21_V2/Features_" + electrodo +'.csv', index = False)

In [ ]:
minimos = []
maximos = []
for i in range(len(colores)):
  for num_archivo in range(colores[i]+1):
    datos = leerArchivo(paths[i], str(num_archivo))['EEG.O1'].to_list()
    frecuencia, spectrum = signal.periodogram(datos, 128)

In [ ]:
np.where(frecuencia<6.5)

TypeError: ignored